# Redis Search Scoring Demo
This notebook demonstrates usage of the SCORE_FIELD parameter to use a custom document-level score to influence the standard field-level scoring.  Examples are provided for TFIDF and BM25 scoring.  

## Install redis-py Client

In [73]:
! pip install redis

## Start the Redis Stack Docker container

In [74]:
! docker run -d --name redis-stack -p 6379:6379 redis/redis-stack-server:latest

62044f2d75eeb035807e6576b6b9e948bd1c45806bc1e6526501e4425de47263


## CLI - Create Indices and Load JSON Data Item
Below are the commands to create two indices:  
- idx1 is without the SCORE_FIELD param.  Searches against this index yield default scoring.
- idx2 includes SCORE_FIELD.  Searches against index yield the product of the TFIDF or BM25 score and the defined SCORE_FIELD attribute.

In [75]:
! redis-cli flushdb
! redis-cli FT.CREATE idx1 ON JSON PREFIX 1 doc: SCHEMA $.content AS content TEXT
! redis-cli FT.CREATE idx2 ON JSON PREFIX 1 doc: SCORE_FIELD $.doc_score SCHEMA $.content AS content TEXT
! redis-cli JSON.SET doc:1 $ '{"content": "The quick brown fox jumps over the lazy dog",  "doc_score": 0.1}'

OK
OK
OK
OK


## CLI - Default Search Scoring
TFIDF and BM25 searches with scoring below.

In [76]:
! redis-cli FT.SEARCH idx1 @content:"brown fox" SCORER TFIDF WITHSCORES
! redis-cli FT.SEARCH idx1 @content:"brown fox" SCORER BM25 WITHSCORES

1) (integer) 1
2) "doc:1"
3) "2"
4) 1) "$"
   2) "{\"content\":\"The quick brown fox jumps over the lazy dog\",\"doc_score\":0.1}"
1) (integer) 1
2) "doc:1"
3) "0.34482757486713095"
4) 1) "$"
   2) "{\"content\":\"The quick brown fox jumps over the lazy dog\",\"doc_score\":0.1}"


## CLI - Search Scoring Utilizing SCORE_FIELD
The same TFIDF and BM25 searches below but utilizing the index (idx2) with the SCORE_FIELD param.  Note the scores are now multiplied by that param value - 0.1.

In [77]:
! redis-cli FT.SEARCH idx2 @content:"brown fox" SCORER TFIDF WITHSCORES
! redis-cli FT.SEARCH idx2 @content:"brown fox" SCORER BM25 WITHSCORES

1) (integer) 1
2) "doc:1"
3) "0.20000000298023224"
4) 1) "$"
   2) "{\"content\":\"The quick brown fox jumps over the lazy dog\",\"doc_score\":0.1}"
1) (integer) 1
2) "doc:1"
3) "0.034482758000546225"
4) 1) "$"
   2) "{\"content\":\"The quick brown fox jumps over the lazy dog\",\"doc_score\":0.1}"


## Python - Create Indices and Load JSON Data Item
Below are the commands to create two indices:  
- idx1 is without the SCORE_FIELD param.  Searches against this index yield default scoring.
- idx2 includes the SCORE_FIELD.  Searches against this index yield the product of the TFIDF or BM25 score and the defined SCORE_FIELD attribute.

In [78]:
import redis
from redis.commands.search.field import TextField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

client = redis.Redis()
client.flushdb()

schema = [ TextField('$.content', as_name='content') ]
idx1_def = IndexDefinition(index_type=IndexType.JSON, prefix=['doc:'])
idx2_def = IndexDefinition(index_type=IndexType.JSON, prefix=['doc:'], score_field='$.doc_score')

client.ft('idx1').create_index(schema, definition=idx1_def)
client.ft('idx2').create_index(schema, definition=idx2_def)

client.json().set('doc:1', '$', {'content': 'The quick brown fox jumps over the lazy dog', 'doc_score': 0.1})

True

## Python - Default Search Scoring
TFIDF and BM25 searches with scoring below.

In [79]:
tfidf_query = Query('@content:"brown fox"')\
    .scorer('TFIDF')\
    .with_scores()
results = client.ft('idx1').search(tfidf_query)
print('*** TFIDF ***')
print(results)

bm25_query = Query('@content:"brown fox"')\
    .scorer('BM25')\
    .with_scores()
results = client.ft('idx1').search(bm25_query)
print('\n*** BM25 ***')
print(results)

*** TFIDF ***
Result{1 total, docs: [Document {'id': 'doc:1', 'payload': None, 'score': 2.0, 'json': '{"content":"The quick brown fox jumps over the lazy dog","doc_score":0.1}'}]}

*** BM25 ***
Result{1 total, docs: [Document {'id': 'doc:1', 'payload': None, 'score': 0.34482757486713095, 'json': '{"content":"The quick brown fox jumps over the lazy dog","doc_score":0.1}'}]}


## Python - Search Scoring Utilizing SCORE_FIELD
The same TFIDF and BM25 searches below but utilizing the index (idx2) with the SCORE_FIELD param.  Note the scores are now multiplied by that param value - 0.1.

In [80]:
results = client.ft('idx2').search(tfidf_query)
print('*** TFIDF ***')
print(results)

results = client.ft('idx2').search(bm25_query)
print('\n*** BM25 ***')
print(results)

*** TFIDF ***
Result{1 total, docs: [Document {'id': 'doc:1', 'payload': None, 'score': 0.20000000298023224, 'json': '{"content":"The quick brown fox jumps over the lazy dog","doc_score":0.1}'}]}

*** BM25 ***
Result{1 total, docs: [Document {'id': 'doc:1', 'payload': None, 'score': 0.034482758000546225, 'json': '{"content":"The quick brown fox jumps over the lazy dog","doc_score":0.1}'}]}


## Shutdown and Remove the Redis Stack Docker container

In [81]:
! docker container rm -f redis-stack

redis-stack
